This program gives an output of all monsters with a Challenge Rating between x and y (which are user inputs).
This is based on the D&D 5e API available here: http://www.dnd5eapi.co/

In [132]:
import requests
import pprint
import pandas as pd

response = requests.get("http://dnd5eapi.co/api/monsters/")
data = response.json()
results = data["results"]
#This block converts the API into the variable results, which spits out monsters data in JSON format
                               
cr_list = [] #pre-allocate list of all challenge ratings
names_list = [] # pre-allocate list
count = data['count'] #data['count'] # gets the whole monster list length

#This whole for loop 
for i in range(0, count):
    names_i = results[i]['name']
    names_list.append(names_i)
    url_i = results[i]['url']
    monster_data_raw = requests.get(url_i)
    monster_data = monster_data_raw.json()
    cr_list.append(monster_data["challenge_rating"]) 
#now names_list and cr_list are filled with monster names and challenge ratings
    
# print(list(zip(names_list, cr_list)))
dd_df = pd.DataFrame.from_records(zip(names_list, cr_list))
#This uses pandas to put names and crs into a dataframe with rows and columns
dd_df.columns =["Name", "CR"]
dd_df.index += 1 
#Renames the columns from their default of 0 and 1
dd_df.head(n=3)
#This just returns the first 3 rows to make sure everything looks good


,Name,CR
1,Aboleth,10.00
2,Acolyte,0.25
3,Adult Black Dragon,14.00


In [133]:
def get_inputs():
#This function gets the minimum and maxium from the user and returns the resulting range
    min_cr = int(input("Type a minimum between 0 and 30: "))
    max_cr = int(input("Type a maximum between 0 and 30: "))
    return (min_cr, max_cr)

range_cr = get_inputs()
min_cr = range_cr[0]
max_cr = range_cr[1]
#This runs get inputs (getting the range) then re-defines min_cr and max_cr outside of the function
#Now I can use these inputs in the dataframe for a tidy panda output

print("Here are all monsters with a CR between "+str(min_cr)+" and "+str(max_cr)+":")
dd_df[(dd_df["CR"]>min_cr)&(dd_df["CR"]<=max_cr)]
#dd_df is that dataframe based on cr_list and names_list - we're pulling it up within the range

Type a minimum between 0 and 30: 4
Type a maximum between 0 and 30: 10
Here are all monsters with a CR between 4 and 10:


,Name,CR
1,Aboleth,10.0
14,Air Elemental,5.0
30,Assassin,8.0


In [134]:
#Optional: print the stats of the monster at a specified index

foo = input("Type the index of the monster you want stats for: ")
specify_monster = requests.get("http://dnd5eapi.co/api/monsters/"+str(foo))
monster_at_index = specify_monster.json()
pprint.pprint(monster_at_index)

Type the index of the monster you want stats for: 1
{'_id': '5bce91465b7768e7920181a5',
 'actions': [{'attack_bonus': 0,
              'desc': 'The aboleth makes three tentacle attacks.',
              'name': 'Multiattack'},
             {'attack_bonus': 9,
              'damage_bonus': 5,
              'damage_dice': '2d6',
              'desc': 'Melee Weapon Attack: +9 to hit, reach 10 ft., one '
                      'target. Hit: 12 (2d6 + 5) bludgeoning damage. If the '
                      'target is a creature, it must succeed on a DC 14 '
                      'Constitution saving throw or become diseased. The '
                      'disease has no effect for 1 minute and can be removed '
                      'by any magic that cures disease. After 1 minute, the '
                      "diseased creature's skin becomes translucent and slimy, "
                      "the creature can't regain hit points unless it is "
                      'underwater, and the disease can be

In [137]:
# Optional: This gets the name of a random monster within the whole dataset
random_monster = dd_df.sample(n=1, random_state=0)
#random state of 1 preserves the output, random state 0 will change the output every run
print(random_monster)

monster_index = random_monster.index.values.astype(int)
#This stores the index of that random monster

                 Name    CR
3  Adult Black Dragon  14.0


In [138]:
#If you then want all of that random monster's stats, printed nicely with the "pretty print" module:
response_random = requests.get("http://dnd5eapi.co/api/monsters/"+str(monster_index[0]))
data_random = response_random.json()
pprint.pprint(data_random)

{'_id': '5bce91465b7768e7920181a4',
 'actions': [{'attack_bonus': 0,
              'desc': 'The dragon can use its Frightful Presence. It then '
                      'makes three attacks: one with its bite and two with its '
                      'claws.',
              'name': 'Multiattack'},
             {'attack_bonus': 11,
              'damage_bonus': 6,
              'damage_dice': '2d10 + 1d8',
              'desc': 'Melee Weapon Attack: +11 to hit, reach 10 ft., one '
                      'target. Hit: 17 (2d10 + 6) piercing damage plus 4 (1d8) '
                      'acid damage.',
              'name': 'Bite'},
             {'attack_bonus': 11,
              'damage_bonus': 6,
              'damage_dice': '2d6',
              'desc': 'Melee Weapon Attack: +11 to hit, reach 5 ft., one '
                      'target. Hit: 13 (2d6 + 6) slashing damage.',
              'name': 'Claw'},
             {'attack_bonus': 11,
              'damage_bonus': 6,
              'damage_di